In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import tokenize
from sklearn import preprocessing
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Plotting tools
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
df = pd.read_excel('C:/Katch_Media/GCP_Data/Reviews/Movie_Book.xlsx')

df

,IMDbID,posts
0,tt0013442,"""CRAS..."
1,tt0015648,Battleship Potemkin\n\nScenario and script by ...
2,tt0017136,METROPOLIS\n\n\n b...
3,tt0019254,A CROWDED ROOM\n\n\n\n\n\n\n\n\n\n\n\n\n ...
4,tt0021749,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
5,tt0021884,FRANKENSTEIN\n \n \n ...
6,tt0022100,"M\n\n\nA large white M on a dark, shadowy scre..."
7,tt0022913,Tod Browning's Freaks - screenplay - Transcrip...
8,tt0023969,Duck Soup...
9,tt0024216,"HE SCREENPLAY OF ""KING KONG""\n\n\n ..."


In [5]:
filepath = ('C:/Katch_Media/GCP_Data/Reviews/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')

In [6]:
    emolex_df = pd.read_csv(filepath,
                            names=["word", "emotion", "association"],
                            sep='\t')

In [7]:
 emolex_words = emolex_df.pivot(index='word',
                                   columns='emotion',
                                   values='association').reset_index()


In [8]:
    emotions = emolex_words.columns.drop('word')
    emo_df = pd.DataFrame(0, index=df.index, columns=emotions)

    stemmer = SnowballStemmer("english")

In [9]:
#Making all letters lowercase
df['posts'] = df['posts'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [10]:
#Removing Punctuation, Symbols
df['posts'] = df['posts'].str.replace('[^\w\s]',' ')

In [11]:
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['posts'] = df['posts'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [12]:
#Lemmatisation
from textblob import Word
df['posts'] = df['posts'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [13]:
#Remove numbers
df['posts'] = df['posts'].str.replace('\d+', '')


In [14]:
for i, row in df.iterrows():
    document = word_tokenize(df.loc[i]['posts'])
    for word in document:
        #word = stemmer.stem(word.lower())
        #print(word)
        emo_score = emolex_words[emolex_words.word == word]
        #print(emo_score)
        if not emo_score.empty:
            for emotion in list(emotions):
                emo_df.at[i, emotion] += emo_score[emotion]
new_df = pd.concat([df, emo_df], axis=1)

In [15]:
new_df_word = new_df
new_df_word

,IMDbID,posts,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,tt0013442,crash screenplay david cronenberg based novel ...,57,69,29,91,29,146,122,74,44,74
1,tt0015648,battleship potemkin scenario script sergei eis...,213,84,82,254,60,340,404,111,82,294
2,tt0017136,metropolis corey mandell fade ext manhattan mi...,98,90,55,140,89,184,214,108,77,146
3,tt0019254,crowded room james cameron fade know looking a...,79,86,54,140,64,145,231,66,45,175
4,tt0021749,doctype html public wc dtd xhtml transitiona...,29,10,30,38,16,33,130,33,0,52
5,tt0021884,frankenstein novel mary w shelley screen story...,141,133,75,177,159,275,275,125,87,127
6,tt0022100,large white dark shadowy screen music hall mou...,125,145,97,213,105,229,308,140,62,219
7,tt0022913,tod browning freak screenplay transcript cult...,80,162,72,96,178,173,273,75,125,142
8,tt0023969,duck soup screenplay harry ruby bert kalmar g...,26,60,11,37,69,93,194,34,40,151
9,tt0024216,screenplay king kong k n g change fade c...,79,134,40,96,88,145,183,65,93,115


In [26]:
new_df_word['fear']

posts
crash screenplay david cronenberg based novel j g ballard shooting draft ext airfield day moving small airfield full parked light plane people around move cluster plane toward hangar edge field int hangar day still moving light plane inside hangar plane engine cover open part strewn around others partially covered tarp section missing even sleek executive jet parked one corner float past plane notice woman leaning wing piper cub chest wing trailing edge arm spread side though flying get closer see jacket pulled open expose one breast rest metal wing cu breast metal cu hard nipple rivet cu woman catherine early thirty dark short hair stylish executive clothes eye wide open unfocussed hand grip shoulder behind follow hand behind catherine discover man crouched behind kissing back catherine standing low mechanic platform skirt raised hooked wing flap wear garter stocking panty man handsome cruel looking rise behind enters kiss neck catherine half close eye rotates pelvis gently thru

In [16]:
new_df_word.to_csv('C:/Katch_Media/GCP_Data/Reviews/movie_word_freq.csv')

In [17]:
new_df['word_count'] = new_df['posts'].apply(tokenize.word_tokenize).apply(len)

In [18]:
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']


In [19]:
for emotion in emotions:
    new_df[emotion] = new_df[emotion] / new_df['word_count']

In [20]:
new_df.set_index('posts', inplace=True)

In [21]:
new_df

IMDbID     anger  \
posts                                                                     
crash screenplay david cronenberg based novel j...  tt0013442  0.027996   
battleship potemkin scenario script sergei eise...  tt0015648  0.073196   
metropolis corey mandell fade ext manhattan mid...  tt0017136  0.036581   
crowded room james cameron fade know looking ab...  tt0019254  0.032972   
doctype html public wc dtd xhtml   transitional...  tt0021749  0.012299   
frankenstein novel mary w shelley screen story ...  tt0021884  0.055930   
large white dark shadowy screen music hall moun...  tt0022100  0.042201   
tod browning freak screenplay transcript  cult ...  tt0022913  0.026290   
duck soup  screenplay harry ruby bert kalmar gr...  tt0023969  0.017711   
screenplay king kong k n g change       fade co...  tt0024216  0.032807   
pump volume transcript happy harry hardon ever ...  tt0102926  0.050696   
pre romeo juliet william shakespeare anchor wom...  tt1343092  0.054622   
clue script clue transcript transcript clue  mo...  tt7286456  0.032616   
  december   mr menahem golan cannon film  suns...  tt0145487  0.038878   
doctype html public wc dtd xhtml   transitional...  tt0816692  0.012299   
source code written ben ripley    source code d...  tt0945513  0.032392   
scary movie  writer shawn wayans marlon wayans ...  tt0993846  0.071429   
bounty hunter written sarah thorp sept   first ...  tt1038919  0.037657   
city joy written bygerald brach roland joffe oc...  tt1058017  0.019798   
cruel intention roger kumble cruel invention ro...  tt1092026  0.030681   
burlesque written steven antin    title sequenc...  tt1126591  0.026915   
frankenweenie written john augustwhirs hum scre...  tt1142977  0.037795   
orphan written david leslie johnson february   ...  tt1148204  0.022035   
jane eyre written moira buffini adapted novel c...  tt1229822  0.043566   
looper written rian johnson production draft ex...  tt1276104  0.038305   
alien project formerly titled starbeast story d...  tt1401152  0.030361   
agnes godfade ext convent daywhispering nun pra...  tt1403865  0.022539   
buried written chris sparling fade int unknown ...  tt1462758  0.036355   
fade fade black white dreamycu serene face beat...  tt1499658  0.021231   
 hour written simon beaufoy amp danny boyle ext...  tt1542344  0.029084   

                                                    anticipation   disgust  \
posts                                                                        
crash screenplay david cronenberg based novel j...      0.033890  0.014244   
battleship potemkin scenario script sergei eise...      0.028866  0.028179   
metropolis corey mandell fade ext manhattan mid...      0.033595  0.020530   
crowded room james cameron fade know looking ab...      0.035893  0.022538   
doctype html public wc dtd xhtml   transitional...      0.004241  0.012723   
frankenstein novel mary w shelley screen story ...      0.052757  0.029750   
large white dark shadowy screen music hall moun...      0.048953  0.032748   
tod browning freak screenplay transcript  cult ...      0.053237  0.023661   
duck soup  screenplay harry ruby bert kalmar gr...      0.040872  0.007493   
screenplay king kong k n g change       fade co...      0.055648  0.016611   
pump volume transcript happy harry hardon ever ...      0.064281  0.023194   
pre romeo juliet william shakespeare anchor wom...      0.075630  0.041550   
clue script clue transcript transcript clue  mo...      0.045336  0.019896   
  december   mr menahem golan cannon film  suns...      0.034493  0.030400   
doctype html public wc dtd xhtml   transitional...      0.004241  0.012723   
source code written ben ripley    source code d...      0.033318  0.015733   
scary movie  writer shawn wayans marlon wayans ...      0.000000  0.071429   
bounty hunter written sarah thorp sept   first ...      0.036611  0.027197   
city joy written bygerald brach roland joffe oc...      0.050803  0.017931   
cruel intention 

In [21]:
new_df.describe()

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,word_count
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.034908,0.040509,0.025335,0.043410,0.038736,0.073068,0.091979,0.037025,0.027085,0.053697,2353.500000
std,0.014839,0.018337,0.014265,0.017049,0.023749,0.023667,0.028857,0.014860,0.013094,0.025624,643.500221
min,0.012299,0.000000,0.007493,0.016115,0.000000,0.013995,0.050902,0.013995,0.000000,0.000000,28.000000
25%,0.026446,0.033387,0.015953,0.032174,0.022331,0.061226,0.074246,0.027338,0.021689,0.037645,2066.000000
50%,0.032712,0.040726,0.022866,0.040311,0.035997,0.070055,0.088898,0.033557,0.025716,0.048907,2402.000000
75%,0.038734,0.050341,0.029357,0.051603,0.052982,0.087879,0.103667,0.045607,0.030719,0.069703,2678.500000
max,0.073196,0.083630,0.071429,0.087285,0.106443,0.120061,0.172269,0.071429,0.071429,0.103641,3421.000000


In [22]:
new_df_prob = new_df
#new_df_prob.to_csv('C:/Katch_Media/GCP_Data/Reviews/movie_prob_freq.csv')

In [23]:
df_model = pd.read_csv('C:/Katch_Media/GCP_Data/Reviews/data/moodyData - Model.csv')
df_model


,Unnamed: 0,IMDbID,title,mood_emotion_--_Anger,mood_emotion_--_Creepiness,mood_emotion_--_Disgust,mood_emotion_--_Empowerment,mood_emotion_--_Fear,mood_emotion_--_Joy / Happiness,mood_emotion_--_Nostalgia / Sentimentality (Tug at Heartstrings),...,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,word_count
0,0,tt0093822,Raising Arizona (1987),1.0,1.5,0.0,3.0,0.0,4.0,5.0,...,0.101670,0.042731,0.134086,0.042731,0.215128,0.179764,0.109037,0.064833,0.109037,2036
1,1,tt0098635,WHEN HARRY MET SALLY (1989),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.086598,0.084536,0.261856,0.061856,0.350515,0.416495,0.114433,0.084536,0.303093,2910
2,2,tt1343092,THE GREAT GATSBY (2013),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.100784,0.061590,0.156775,0.099664,0.206047,0.239642,0.120941,0.086226,0.163494,2679
3,3,tt1457767,The Conjuring (2015),3.0,2.0,3.0,1.0,0.0,3.0,0.0,...,0.107679,0.067613,0.175292,0.080134,0.181553,0.289232,0.082638,0.056344,0.219115,2396
4,4,tt3065204,The Conjuring 2 (Jan 2015),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012723,0.038168,0.048346,0.020356,0.041985,0.165394,0.041985,0.000000,0.066158,2358
5,5,tt7286456,JOKER (2019),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.158271,0.089250,0.210631,0.189211,0.327251,0.327251,0.148750,0.103530,0.151131,2521
6,6,tt8079248,YESTERDAY (2019),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.146860,0.098244,0.215733,0.106347,0.231938,0.311951,0.141796,0.062795,0.221810,2962
7,7,tt0458352,THE DEVIL WEARS PRADA (2006),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.159711,0.070983,0.094643,0.175485,0.170555,0.269142,0.073940,0.123234,0.139993,3043
8,8,tt0102926,SILENCE OF THE LAMBS (1992),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.122616,0.022480,0.075613,0.141008,0.190054,0.396458,0.069482,0.081744,0.308583,1468
9,9,tt2671706,FENCES (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.166944,0.049834,0.119601,0.109635,0.180648,0.227990,0.080980,0.115864,0.143272,2408


In [24]:
df_model_normalize = df_model.iloc[:,3:17]
df_model_normalize

,mood_emotion_--_Anger,mood_emotion_--_Creepiness,mood_emotion_--_Disgust,mood_emotion_--_Empowerment,mood_emotion_--_Fear,mood_emotion_--_Joy / Happiness,mood_emotion_--_Nostalgia / Sentimentality (Tug at Heartstrings),mood_emotion_--_Peacefulness,mood_emotion_--_Sadness / Sorrow,mood_emotion_--_Surprise,mood_emotion_--_Tenderness / Sweetness,mood_emotion_--_Tension,mood_emotion_--_Transcendence,mood_emotion_--_Wonder
0,1.0,1.5,0.0,3.0,0.0,4.0,5.0,0.0,0.0,3.0,2.5,1.5,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2.0,3.0,1.0,0.0,3.0,0.0,2.0,3.0,0.0,2.0,3.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:


x = df_model_normalize.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_normalized = pd.DataFrame(x_scaled)

df_normalized.rename(columns={0:'mood_Anger',
                          1:'mood_Creepiness',
                          2:'mood_Disgust',
                          3:'mood_Empowerment',
                          4:'mood_Fear',
                          5:'mood_Joy',
                          6:'mood_Nostalgia',
                          7:'mood_Peacefulness',
                          8:'mood_Sadness',
                          9:'mood_Surprise',
                          10:'mood_Tenderness',
                          11:'mood_Tension',
                          12:'mood_Transcendence',
                          13:'mood_Wonder'}, inplace=True)
df_normalized

,mood_Anger,mood_Creepiness,mood_Disgust,mood_Empowerment,mood_Fear,mood_Joy,mood_Nostalgia,mood_Peacefulness,mood_Sadness,mood_Surprise,mood_Tenderness,mood_Tension,mood_Transcendence,mood_Wonder
0,0.333333,0.6,0.000000,0.750,0.00,1.000,1.0,0.000000,0.000000,1.000000,0.625,0.500000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
3,1.000000,0.8,1.000000,0.250,0.00,0.750,0.0,0.444444,0.857143,0.000000,0.500,1.000000,0.666667,0.000000
4,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
5,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
8,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,0.000000,0.0,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000


In [26]:
df_left = df_model.drop(df_model.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]], axis=1)
df_left

,Unnamed: 0,IMDbID,title,posts,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,word_count
0,0,tt0093822,Raising Arizona (1987),crash screenplay david cronenberg based novel ...,0.083988,0.101670,0.042731,0.134086,0.042731,0.215128,0.179764,0.109037,0.064833,0.109037,2036
1,1,tt0098635,WHEN HARRY MET SALLY (1989),battleship potemkin scenario script sergei eis...,0.219588,0.086598,0.084536,0.261856,0.061856,0.350515,0.416495,0.114433,0.084536,0.303093,2910
2,2,tt1343092,THE GREAT GATSBY (2013),metropolis corey mandell fade ext manhattan mi...,0.109742,0.100784,0.061590,0.156775,0.099664,0.206047,0.239642,0.120941,0.086226,0.163494,2679
3,3,tt1457767,The Conjuring (2015),crowded room james cameron fade know looking a...,0.098915,0.107679,0.067613,0.175292,0.080134,0.181553,0.289232,0.082638,0.056344,0.219115,2396
4,4,tt3065204,The Conjuring 2 (Jan 2015),doctype html public wc dtd xhtml transitiona...,0.036896,0.012723,0.038168,0.048346,0.020356,0.041985,0.165394,0.041985,0.000000,0.066158,2358
5,5,tt7286456,JOKER (2019),frankenstein novel mary w shelley screen story...,0.167791,0.158271,0.089250,0.210631,0.189211,0.327251,0.327251,0.148750,0.103530,0.151131,2521
6,6,tt8079248,YESTERDAY (2019),large white dark shadowy screen music hall mou...,0.126604,0.146860,0.098244,0.215733,0.106347,0.231938,0.311951,0.141796,0.062795,0.221810,2962
7,7,tt0458352,THE DEVIL WEARS PRADA (2006),tod browning freak screenplay transcript cult...,0.078870,0.159711,0.070983,0.094643,0.175485,0.170555,0.269142,0.073940,0.123234,0.139993,3043
8,8,tt0102926,SILENCE OF THE LAMBS (1992),duck soup screenplay harry ruby bert kalmar g...,0.053134,0.122616,0.022480,0.075613,0.141008,0.190054,0.396458,0.069482,0.081744,0.308583,1468
9,9,tt2671706,FENCES (2016),screenplay king kong k n g change fade c...,0.098422,0.166944,0.049834,0.119601,0.109635,0.180648,0.227990,0.080980,0.115864,0.143272,2408


In [232]:
df_final = pd.concat([df_left, df_normalized], axis=1, sort=False)
df_final.iloc[19:,:]

,Unnamed: 0,IMDbID,title,posts,anger,anticipation,disgust,fear,joy,negative,...,mood_Fear,mood_Joy,mood_Nostalgia,mood_Peacefulness,mood_Sadness,mood_Surprise,mood_Tenderness,mood_Tension,mood_Transcendence,mood_Wonder
19,19,tt0092890,Dirty Dancing (1987),cruel intention roger kumble cruel invention r...,0.092044,0.135725,0.071763,0.090484,0.126365,0.191888,...,0.75,0.625,0.0,0.333333,0.285714,0.000000,0.625,0.833333,0.0,0.666667
20,20,tt5247022,Paterson (2016),burlesque written steven antin title sequen...,0.080745,0.121739,0.040994,0.075776,0.129193,0.180124,...,0.50,0.375,0.5,1.000000,0.571429,0.000000,0.625,0.000000,0.0,0.000000
21,21,tt4729430,Klaus (2019),frankenweenie written john augustwhirs hum scr...,0.113386,0.095669,0.069685,0.107480,0.198425,0.230315,...,1.00,1.000,0.8,0.666667,1.000000,0.666667,1.000,0.833333,0.0,1.000000
22,22,tt5304992,Set it Up (2018),orphan written david leslie johnson february ...,0.066106,0.159856,0.147837,0.102163,0.194712,0.260817,...,0.00,0.875,0.6,0.444444,0.571429,0.500000,0.750,0.500000,0.0,0.000000
23,23,tt1663202,The Revenant (2015),jane eyre written moira buffini adapted novel ...,0.130699,0.124620,0.177812,0.170213,0.089666,0.360182,...,1.00,0.000,0.0,0.333333,1.000000,0.666667,0.375,1.000000,0.0,0.666667
24,24,tt9358052,Knock Down The House (2019),looper written rian johnson production draft e...,0.114914,0.094132,0.050122,0.110024,0.068460,0.202934,...,0.00,0.875,0.8,0.555556,0.714286,1.000000,0.625,1.000000,1.0,0.000000
25,25,tt8473630,Mononoke Hime (1997),alien project formerly titled starbeast story ...,0.091084,0.143133,0.099759,0.148916,0.066506,0.264578,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000
26,26,tt1568911,War Horse (2011),agnes godfade ext convent daywhispering nun pr...,0.067616,0.250890,0.042705,0.117438,0.249110,0.282918,...,0.50,0.750,0.6,0.333333,0.857143,0.500000,1.000,0.833333,0.0,0.500000
27,27,tt0088247,The Terminator (1984),buried written chris sparling fade int unknown...,0.109065,0.075071,0.038244,0.174221,0.053824,0.276204,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000
28,28,tt3783958,La La Land (2016),fade fade black white dreamycu serene face bea...,0.063694,0.143767,0.052775,0.094631,0.117379,0.162875,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000


In [28]:
#df_final.to_csv('C:/Katch_Media/GCP_Data/Reviews/data/df_final.csv')

In [29]:
df_final.columns

Index(['Unnamed: 0', 'IMDbID', 'title', 'posts', 'anger', 'anticipation',
       'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise',
       'trust', 'word_count', 'mood_Anger', 'mood_Creepiness', 'mood_Disgust',
       'mood_Empowerment', 'mood_Fear', 'mood_Joy', 'mood_Nostalgia',
       'mood_Peacefulness', 'mood_Sadness', 'mood_Surprise', 'mood_Tenderness',
       'mood_Tension', 'mood_Transcendence', 'mood_Wonder'],
      dtype='object')

In [30]:
df_final = df_final[['IMDbID','anger', 'anticipation',
       'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise',
       'trust', 'mood_Anger', 'mood_Creepiness', 'mood_Disgust',
       'mood_Empowerment', 'mood_Fear', 'mood_Joy',
       'mood_Nostalgia',
       'mood_Peacefulness', 'mood_Sadness', 'mood_Surprise',
       'mood_Tenderness', 'mood_Tension', 'mood_Transcendence',
       'mood_Wonder']]
df_final

,IMDbID,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,...,mood_Fear,mood_Joy,mood_Nostalgia,mood_Peacefulness,mood_Sadness,mood_Surprise,mood_Tenderness,mood_Tension,mood_Transcendence,mood_Wonder
0,tt0093822,0.083988,0.101670,0.042731,0.134086,0.042731,0.215128,0.179764,0.109037,0.064833,...,0.00,1.000,1.0,0.000000,0.000000,1.000000,0.625,0.500000,0.000000,0.000000
1,tt0098635,0.219588,0.086598,0.084536,0.261856,0.061856,0.350515,0.416495,0.114433,0.084536,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,tt1343092,0.109742,0.100784,0.061590,0.156775,0.099664,0.206047,0.239642,0.120941,0.086226,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
3,tt1457767,0.098915,0.107679,0.067613,0.175292,0.080134,0.181553,0.289232,0.082638,0.056344,...,0.00,0.750,0.0,0.444444,0.857143,0.000000,0.500,1.000000,0.666667,0.000000
4,tt3065204,0.036896,0.012723,0.038168,0.048346,0.020356,0.041985,0.165394,0.041985,0.000000,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
5,tt7286456,0.167791,0.158271,0.089250,0.210631,0.189211,0.327251,0.327251,0.148750,0.103530,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,tt8079248,0.126604,0.146860,0.098244,0.215733,0.106347,0.231938,0.311951,0.141796,0.062795,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,tt0458352,0.078870,0.159711,0.070983,0.094643,0.175485,0.170555,0.269142,0.073940,0.123234,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
8,tt0102926,0.053134,0.122616,0.022480,0.075613,0.141008,0.190054,0.396458,0.069482,0.081744,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,tt2671706,0.098422,0.166944,0.049834,0.119601,0.109635,0.180648,0.227990,0.080980,0.115864,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000


In [31]:
df_final.columns

Index(['IMDbID', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust', 'mood_Anger',
       'mood_Creepiness', 'mood_Disgust', 'mood_Empowerment', 'mood_Fear',
       'mood_Joy', 'mood_Nostalgia', 'mood_Peacefulness', 'mood_Sadness',
       'mood_Surprise', 'mood_Tenderness', 'mood_Tension',
       'mood_Transcendence', 'mood_Wonder'],
      dtype='object')

In [32]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,12]


from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [33]:
glm_mod0 = smf.glm(formula = "mood_Anger ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod0.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)
C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:735: RuntimeWarning: divide by zero encountered in log
  ll_obs -= special.gammaln(weight_scale) + np.log(endog)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             mood_Anger   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          50.072
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1566.5
Time:                        09:55:19   Pearson chi2:                     951.
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -67.5746    149.707     -0.451      0.652    -360.996     225.847
anger          299.2642   1136.970      0.263      0.792   -1929.157    2527.685
anticipation   191.7361    724.023      0.265      0.791   -1227.323    1610.795
disgust       -200.5412    698.011     -0.287      0.774   -1568.619    1167.536
fear          -185.8018    826.774     -0.225      0.822   -1806.248    1434.645
joy           -787.7976   1332.195     -0.591      0.554   -3398.852    1823.257
negative      -400.1879    818.467     -0.489      0.625   -2004.354    1203.978
positive       860.7348   1471.513      0.585      0.559   -2023.377    3744.847
sadness        746.0990   1268.354      0.588      0.556   -1739.828    3232.026
surprise       540.8314   1532.188      0.353      0.724   -2462.202    3543.865
trust         -640.3984   1156.304     -0.554      0.580   -2906.712    1625.916
================================================================================
"""

In [34]:
glm_mod1 = smf.glm(formula = "mood_Creepiness ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod1.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:        mood_Creepiness   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          1.3684
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1822.3
Time:                        09:55:19   Pearson chi2:                     26.0
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -2926.6482   7800.921     -0.375      0.708   -1.82e+04    1.24e+04
anger         6.514e+05   1.02e+05      6.356      0.000    4.51e+05    8.52e+05
anticipation  2.861e+05   6.76e+04      4.235      0.000    1.54e+05    4.18e+05
disgust       6.666e+04   3.03e+04      2.197      0.028    7188.951    1.26e+05
fear         -6.807e+05   8.95e+04     -7.608      0.000   -8.56e+05   -5.05e+05
joy          -2.716e+05   6.81e+04     -3.986      0.000   -4.05e+05   -1.38e+05
negative     -2.509e+05   5.45e+04     -4.599      0.000   -3.58e+05   -1.44e+05
positive      1.819e+05   7.12e+04      2.557      0.011    4.25e+04    3.21e+05
sadness       3.486e+05   5.02e+04      6.947      0.000     2.5e+05    4.47e+05
surprise     -4.767e+04   1.02e+05     -0.466      0.641   -2.48e+05    1.53e+05
trust         5.055e+04    6.6e+04      0.766      0.444   -7.89e+04     1.8e+05
================================================================================
"""

In [35]:
glm_mod2 = smf.glm(formula = "mood_Disgust ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod2.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           mood_Disgust   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          18.270
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1718.1
Time:                        09:55:19   Pearson chi2:                     347.
No. Iterations:                    15                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -1515.7258   1567.327     -0.967      0.334   -4587.630    1556.178
anger          206.4546    655.585      0.315      0.753   -1078.469    1491.378
anticipation  3534.2109   3935.660      0.898      0.369   -4179.542    1.12e+04
disgust      -3968.2865   4055.314     -0.979      0.328   -1.19e+04    3979.983
fear           987.1571   1226.900      0.805      0.421   -1417.523    3391.837
joy          -1.321e+04   1.34e+04     -0.987      0.324   -3.95e+04     1.3e+04
negative     -6946.9655   7015.619     -0.990      0.322   -2.07e+04    6803.394
positive      1.479e+04    1.5e+04      0.988      0.323   -1.45e+04    4.41e+04
sadness       1.225e+04   1.22e+04      1.002      0.316   -1.17e+04    3.62e+04
surprise       1.62e+04   1.65e+04      0.984      0.325   -1.61e+04    4.85e+04
trust        -1.219e+04   1.23e+04     -0.988      0.323   -3.64e+04     1.2e+04
================================================================================
"""

In [36]:
glm_mod3 = smf.glm(formula = "mood_Empowerment ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod3.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       mood_Empowerment   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          11.006
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1572.5
Time:                        09:55:20   Pearson chi2:                     209.
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -30.3886     50.863     -0.597      0.550    -130.079      69.302
anger          139.5449    354.005      0.394      0.693    -554.293     833.382
anticipation   255.2372    402.679      0.634      0.526    -533.998    1044.473
disgust        136.7866    211.773      0.646      0.518    -278.282     551.855
fear            -9.7102    411.784     -0.024      0.981    -816.792     797.372
joy           -428.9102    485.017     -0.884      0.377   -1379.525     521.705
negative      -406.8709    358.976     -1.133      0.257   -1110.451     296.709
positive       469.5143    493.019      0.952      0.341    -496.786    1435.815
sadness        297.3922    362.533      0.820      0.412    -413.160    1007.944
surprise       327.6636    627.563      0.522      0.602    -902.338    1557.665
trust         -323.4752    414.468     -0.780      0.435   -1135.818     488.868
================================================================================
"""

In [37]:
glm_mod4 = smf.glm(formula = "mood_Joy ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod4.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               mood_Joy   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          10.603
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1571.6
Time:                        09:55:21   Pearson chi2:                     201.
No. Iterations:                    14                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -75.5030     67.546     -1.118      0.264    -207.890      56.884
anger          543.1127    447.216      1.214      0.225    -333.415    1419.640
anticipation   318.8987    259.305      1.230      0.219    -189.330     827.127
disgust        321.0680    248.952      1.290      0.197    -166.868     809.004
fear          -478.5875    388.355     -1.232      0.218   -1239.749     282.574
joy           -718.9568    545.662     -1.318      0.188   -1788.434     350.520
negative      -167.9610    205.332     -0.818      0.413    -570.404     234.482
positive       677.1173    539.138      1.256      0.209    -379.574    1733.808
sadness        384.8290    358.438      1.074      0.283    -317.697    1087.355
surprise      -100.2395    234.383     -0.428      0.669    -559.621     359.142
trust         -370.6535    327.623     -1.131      0.258   -1012.782     271.475
================================================================================
"""

In [38]:
glm_mod5 = smf.glm(formula = "mood_Nostalgia ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod5.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         mood_Nostalgia   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          1.2632
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1682.1
Time:                        09:55:21   Pearson chi2:                     24.0
No. Iterations:                    14                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -3.408e+04   9807.743     -3.475      0.001   -5.33e+04   -1.49e+04
anger         2.071e+06   2.39e+05      8.664      0.000     1.6e+06    2.54e+06
anticipation  2.748e+06   3.23e+05      8.497      0.000    2.11e+06    3.38e+06
disgust       1.995e+06   2.33e+05      8.568      0.000    1.54e+06    2.45e+06
fear          3.151e+05   8.41e+04      3.748      0.000     1.5e+05     4.8e+05
joy          -2.239e+06   2.63e+05     -8.504      0.000   -2.75e+06   -1.72e+06
negative     -2.591e+06   3.05e+05     -8.499      0.000   -3.19e+06   -1.99e+06
positive      2.231e+06   2.62e+05      8.526      0.000    1.72e+06    2.74e+06
sadness      -3.369e+05    8.6e+04     -3.918      0.000   -5.05e+05   -1.68e+05
surprise     -2.284e+06   2.77e+05     -8.240      0.000   -2.83e+06   -1.74e+06
trust        -1.009e+06   1.42e+05     -7.102      0.000   -1.29e+06    -7.3e+05
================================================================================
"""

In [39]:
glm_mod6 = smf.glm(formula = "mood_Peacefulness ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod6.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:      mood_Peacefulness   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          11.280
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1572.8
Time:                        09:55:21   Pearson chi2:                     214.
No. Iterations:                    13                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -8.0854     55.394     -0.146      0.884    -116.656     100.485
anger           53.6962    376.755      0.143      0.887    -684.730     792.122
anticipation  -148.4284    381.270     -0.389      0.697    -895.704     598.847
disgust        156.7215    251.624      0.623      0.533    -336.452     649.895
fear           -78.6533    487.623     -0.161      0.872   -1034.377     877.070
joy           -294.6027    424.516     -0.694      0.488   -1126.639     537.434
negative      -121.8244    359.194     -0.339      0.734    -825.831     582.183
positive       347.6996    427.930      0.813      0.416    -491.027    1186.426
sadness        427.6887    460.104      0.930      0.353    -474.098    1329.476
surprise      -264.5545    386.016     -0.685      0.493   -1021.131     492.022
trust         -145.0110    316.727     -0.458      0.647    -765.785     475.763
================================================================================
"""

In [40]:
glm_mod7 = smf.glm(formula = "mood_Sadness ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod7.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           mood_Sadness   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          48.266
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1643.2
Time:                        09:55:21   Pearson chi2:                     917.
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -15.0784     86.553     -0.174      0.862    -184.719     154.562
anger          392.6748   1246.805      0.315      0.753   -2051.018    2836.368
anticipation   270.6801    783.729      0.345      0.730   -1265.401    1806.761
disgust          7.6058    553.894      0.014      0.989   -1078.007    1093.219
fear          -400.3452    841.972     -0.475      0.634   -2050.580    1249.890
joy           -683.7182   1247.657     -0.548      0.584   -3129.080    1761.644
negative      -378.7079    883.116     -0.429      0.668   -2109.584    1352.169
positive       698.9316   1337.705      0.522      0.601   -1922.922    3320.785
sadness        550.8072   1106.410      0.498      0.619   -1617.717    2719.332
surprise        31.7660    626.686      0.051      0.960   -1196.516    1260.048
trust         -485.7154   1036.115     -0.469      0.639   -2516.463    1545.032
================================================================================
"""

In [41]:
glm_mod8 = smf.glm(formula = "mood_Surprise ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod8.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          mood_Surprise   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          1.2632
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1682.1
Time:                        09:55:21   Pearson chi2:                     24.0
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -2.632e+04   6177.943     -4.260      0.000   -3.84e+04   -1.42e+04
anger         5.128e+05   6.05e+04      8.482      0.000    3.94e+05    6.31e+05
anticipation  5.703e+05   8.66e+04      6.587      0.000    4.01e+05     7.4e+05
disgust       4.354e+05   5.65e+04      7.701      0.000    3.25e+05    5.46e+05
fear         -6881.8140   4.82e+04     -0.143      0.886   -1.01e+05    8.76e+04
joy          -6.106e+05   7.88e+04     -7.746      0.000   -7.65e+05   -4.56e+05
negative     -6.604e+05      8e+04     -8.251      0.000   -8.17e+05   -5.04e+05
positive      6.212e+05   7.91e+04      7.853      0.000    4.66e+05    7.76e+05
sadness       7.418e+04   5.25e+04      1.414      0.157   -2.86e+04    1.77e+05
surprise     -2.348e+05   9.44e+04     -2.487      0.013    -4.2e+05   -4.98e+04
trust         -2.64e+05   5.96e+04     -4.433      0.000   -3.81e+05   -1.47e+05
================================================================================
"""

In [42]:

glm_mod9 = smf.glm(formula = "mood_Tenderness ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod9.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:        mood_Tenderness   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          6.7869
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1497.2
Time:                        09:55:21   Pearson chi2:                     129.
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.8687     24.765     -0.156      0.876     -52.406      44.669
anger          142.2044    220.654      0.644      0.519    -290.269     574.678
anticipation    63.5163    192.596      0.330      0.742    -313.964     440.997
disgust        101.1190    124.089      0.815      0.415    -142.092     344.330
fear            10.2498    208.972      0.049      0.961    -399.328     419.828
joy           -121.8623    200.711     -0.607      0.544    -515.248     271.524
negative      -193.2268    178.532     -1.082      0.279    -543.143     156.690
positive       128.4175    220.446      0.583      0.560    -303.649     560.485
sadness        110.8785    169.693      0.653      0.513    -221.714     443.471
surprise        -1.7629    212.157     -0.008      0.993    -417.582     414.057
trust          -44.7222    179.152     -0.250      0.803    -395.853     306.409
================================================================================
"""

In [43]:
glm_mod10 = smf.glm(formula = "mood_Transcendence ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod10.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     mood_Transcendence   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          1.4737
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1962.4
Time:                        09:55:22   Pearson chi2:                     28.0
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -1.657e+04   4005.579     -4.136      0.000   -2.44e+04   -8718.066
anger        -1.282e+05   5.82e+04     -2.204      0.028   -2.42e+05   -1.42e+04
anticipation  1.227e+05   4.29e+04      2.857      0.004    3.85e+04    2.07e+05
disgust       7211.7769   5.61e+04      0.128      0.898   -1.03e+05    1.17e+05
fear          -1.02e+05   4.52e+04     -2.258      0.024    -1.9e+05   -1.35e+04
joy          -1.768e+05   5.44e+04     -3.250      0.001   -2.83e+05   -7.02e+04
negative       5.16e+04   4.03e+04      1.281      0.200   -2.73e+04    1.31e+05
positive      3.091e+05   5.05e+04      6.115      0.000     2.1e+05    4.08e+05
sadness       9.336e+04   4.47e+04      2.090      0.037    5824.816    1.81e+05
surprise     -4.177e+04   4.89e+04     -0.854      0.393   -1.38e+05    5.41e+04
trust         -2.58e+05   5.73e+04     -4.502      0.000    -3.7e+05   -1.46e+05
================================================================================
"""

In [44]:
glm_mod11 = smf.glm(formula = "mood_Wonder ~ anger + anticipation + disgust + fear + joy + negative + positive + sadness + surprise + trust",data = df_final,
                  family = sm.families.Gamma()).fit();
glm_mod11.summary()

C:\Users\Chirag\anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:278: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            mood_Wonder   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                   Gamma   Df Model:                           10
Link Function:          inverse_power   Scale:                          1.3684
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 25 Aug 2020   Deviance:                       1822.3
Time:                        09:55:22   Pearson chi2:                     26.0
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     4.137e+04   8270.339      5.002      0.000    2.52e+04    5.76e+04
anger        -3.934e+05   4.81e+04     -8.186      0.000   -4.88e+05   -2.99e+05
anticipation -3.563e+05   7.72e+04     -4.614      0.000   -5.08e+05   -2.05e+05
disgust      -1.764e+05    3.7e+04     -4.762      0.000   -2.49e+05   -1.04e+05
fear          6.643e+05   1.07e+05      6.231      0.000    4.55e+05    8.73e+05
joy           1.721e+05   5.96e+04      2.886      0.004    5.52e+04    2.89e+05
negative     -1.978e+05   5.61e+04     -3.529      0.000   -3.08e+05   -8.79e+04
positive     -2.059e+05   7.73e+04     -2.664      0.008   -3.57e+05   -5.44e+04
sadness       1.797e+05   7.69e+04      2.338      0.019    2.91e+04     3.3e+05
surprise      3.737e+05   7.16e+04      5.218      0.000    2.33e+05    5.14e+05
trust         1.352e+05   6.25e+04      2.163      0.031    1.27e+04    2.58e+05
================================================================================
"""

In [45]:

pred_glm = glm_mod11.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 0.9851851681737441


In [46]:
pred_glm = glm_mod10.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -0.1998079367516894


In [47]:
pred_glm = glm_mod9.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -0.3638647742357497


In [48]:
pred_glm = glm_mod8.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 0.9851850375711785


In [49]:
pred_glm = glm_mod7.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -1.852605387887647


In [50]:
pred_glm = glm_mod6.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -0.8183012590376899


In [51]:
pred_glm = glm_mod5.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -0.19928743330194254


In [52]:
pred_glm = glm_mod4.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -2.306237220222013


In [53]:
pred_glm = glm_mod3.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -0.9855034351321503


In [54]:
pred_glm = glm_mod2.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -5.69493254741118


In [55]:
pred_glm = glm_mod1.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 0.9990517381009504


In [56]:
pred_glm = glm_mod0.predict(x_val);
y_true, y_pred = np.array(y_val), np.array(pred_glm)
print("R2",r2_score(y_val, y_pred))

R2 -2.587771051723179


##### models with mood_surprise

In [219]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,20]

In [220]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [221]:
x_train.shape

(24, 10)

In [222]:
y_train.shape

(24,)

In [223]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions

array([ 0.75310351,  0.28762919,  0.49370201,  0.2223062 ,  0.32260128,
       -0.05382897])

In [224]:
lm.score(x_val, y_val)

-1.2577438244281995

In [225]:
print(lm.coef_)

[-4.75677133 -5.32150206 -3.28659285 -0.35514146  2.39601354  5.14419457
 -1.76212493  1.27019311  1.51907816  0.35886451]


In [226]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [117]:
y_pred = regressor.predict(x_val)
y_pred

array([0.09915731, 0.09863223, 0.11008336, 0.10012907, 0.09780716,
       0.09466599])

In [118]:
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.03367218866242583


In [119]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,20]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [120]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)


array([0.12640307, 0.17068109, 0.04488676, 0.1520754 , 0.03397746,
       0.22675115, 0.15080254, 0.16082974])

In [121]:
regr.score(X_test, y_test)

0.070204762295907

##### models with mood_Wonder

In [84]:
df_final

,IMDbID,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,...,mood_Fear,mood_Joy,mood_Nostalgia,mood_Peacefulness,mood_Sadness,mood_Surprise,mood_Tenderness,mood_Tension,mood_Transcendence,mood_Wonder
0,tt0093822,0.083988,0.101670,0.042731,0.134086,0.042731,0.215128,0.179764,0.109037,0.064833,...,0.00,1.000,1.0,0.000000,0.000000,1.000000,0.625,0.500000,0.000000,0.000000
1,tt0098635,0.219588,0.086598,0.084536,0.261856,0.061856,0.350515,0.416495,0.114433,0.084536,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,tt1343092,0.109742,0.100784,0.061590,0.156775,0.099664,0.206047,0.239642,0.120941,0.086226,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
3,tt1457767,0.098915,0.107679,0.067613,0.175292,0.080134,0.181553,0.289232,0.082638,0.056344,...,0.00,0.750,0.0,0.444444,0.857143,0.000000,0.500,1.000000,0.666667,0.000000
4,tt3065204,0.036896,0.012723,0.038168,0.048346,0.020356,0.041985,0.165394,0.041985,0.000000,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
5,tt7286456,0.167791,0.158271,0.089250,0.210631,0.189211,0.327251,0.327251,0.148750,0.103530,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,tt8079248,0.126604,0.146860,0.098244,0.215733,0.106347,0.231938,0.311951,0.141796,0.062795,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,tt0458352,0.078870,0.159711,0.070983,0.094643,0.175485,0.170555,0.269142,0.073940,0.123234,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
8,tt0102926,0.053134,0.122616,0.022480,0.075613,0.141008,0.190054,0.396458,0.069482,0.081744,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,tt2671706,0.098422,0.166944,0.049834,0.119601,0.109635,0.180648,0.227990,0.080980,0.115864,...,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000


In [233]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,24]

In [234]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [235]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

ValueError: could not convert string to float: 'tt3783958'

In [91]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [93]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.004350561159603417


In [96]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,24]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [97]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.3167925781536409

##### models with mood_Creepiness

In [101]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,12]

In [102]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [103]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

-0.6246812367117303

In [104]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [105]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.007930508340537545


In [108]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,12]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [109]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.12200378857333516

##### models with mood_Nostalgia

In [155]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,17]


In [151]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [152]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

0.0

In [137]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [139]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.0


In [141]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,17]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [144]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.0058953010734617

##### models with mood_Transcendence

In [158]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,23]

In [159]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [160]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

0.0

In [161]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [162]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.0


In [163]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,23]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [164]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.039239900047257636

##### models with mood_Empowerment

In [168]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,14]

In [169]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [170]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

-1.7359449957778499

In [171]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [172]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.0344524797424266


In [173]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,14]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [174]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.20041061609797595

##### models with mood_Tenderness

In [177]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,21]

In [178]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [179]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

-12.50492174468505

In [180]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [181]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 -0.027814173956414612


In [182]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,21]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [183]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.648202021579454

##### models with mood_Joy

In [186]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,16]

In [187]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [191]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

0.0

In [192]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [193]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.0


In [194]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,16]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [195]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.5628813085920457

##### models with mood_Anger

In [202]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,11]

In [203]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [204]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

-0.5702797336834471

In [205]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [206]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 0.006943992469409466


In [207]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,16]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [208]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.5628813085920457

##### models with mood_Disgust

In [210]:
X= df_final.iloc[:,1:11]
y= df_final.iloc[:,13]

In [211]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(X,y,test_size =0.2, random_state=42)

In [212]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
predictions = lm.predict(x_val)
predictions
lm.score(x_val, y_val)

-0.4889616739003775

In [213]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [214]:
y_pred = regressor.predict(x_val)
y_pred
y_true, y_pred = np.array(y_val), np.array(y_pred)
print("R2",r2_score(y_true, y_pred))

R2 -0.03200130400940604


In [217]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

X= df_final.iloc[:,1:11]
y= df_final.iloc[:,16]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
X, y = make_regression(n_samples=500, random_state=1)

In [218]:
regr = MLPRegressor(random_state=1, max_iter=500,hidden_layer_sizes = 10,alpha=0.10).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

-0.5628813085920457